In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

d:\Damodarone\Python\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
d:\Damodarone\Python\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
d:\Damodarone\Python\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please update the gencode to avoid compatibility violations i

In [2]:
## Load the Model
model = load_model('model.h5')

## load the Encoders
with open('encoder/label_encode_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('encoder/ohe_geo.pkl','rb') as file:
    geo_encoder = pickle.load(file)

##Load the Scalar pickle
with open('Scaler/scaler.pkl','rb') as file:
    scaler = pickle.load(file)


In [3]:
input_data = {
    "CreditScore" : 607,
    "Geography" : "France",
    "Gender" : "Male",
    "Age" : 40,
    "Tenure" : 3,
    "Balance" : 50000,
    "NumOfProducts": 2,
    "HasCrCard" : 1,
    "IsActiveMember" : 1,
    "EstimatedSalary" : 60000
}

In [4]:
input_df = pd.DataFrame([input_data])

In [5]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,607,France,Male,40,3,50000,2,1,1,60000


In [6]:
input_df["Gender"] = label_encoder_gender.transform(input_df["Gender"])

In [ ]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,607,France,1,40,3,50000,2,1,1,60000


In [11]:
geo_encoded = geo_encoder.transform([input_df["Geography"]])

d:\Damodarone\Python\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [13]:
geo_encoded.toarray()

array([[1., 0., 0.]])

In [14]:
geo_encoder.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [18]:
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(),columns=geo_encoder.get_feature_names_out())

In [20]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [21]:
df = pd.concat([input_df.drop(columns=["Geography"],axis=1),geo_encoded_df],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,607,1,40,3,50000,2,1,1,60000,1.0,0.0,0.0


In [23]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,607,1,40,3,50000,2,1,1,60000,1.0,0.0,0.0


In [24]:
df_scaled = scaler.transform(df)

In [25]:
df_scaled

array([[-0.46334104,  0.91324755,  0.10479359, -0.69539349, -0.41792108,
         0.80843615,  0.64920267,  0.97481699, -0.70296551,  1.00150113,
        -0.57946723, -0.57638802]])

In [26]:
pred = model.predict(df_scaled)
pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 502ms/step


array([[0.05132449]], dtype=float32)

In [28]:
prediction_proba = pred[0][0]
prediction_proba

np.float32(0.051324487)

In [29]:
if prediction_proba > 0.5:
    print("The Customer is likely to Chrun")
else:
    print("The Customer is not likely to churn the bank")

The Customer is not likely to churn the bank
